In [ ]:
source(paste0(dirname(dirname(getwd())),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

### 0 - Read and process collected driver data
- Just including all drivers output from Linx here

In [ ]:
reader <- function(i) fread(paste0("/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/drivers_full/", i))

In [ ]:
somatic <- reader("somatic.txt")
amp <- reader("amp.txt")
del <- reader("del.txt")
disruption <- reader("disruption.txt")
fusion <- reader("fusion.txt")

- Stack them

In [ ]:
somatic_ready <- somatic %>% se(sampleId, gene, annotation, biallelic, method)
disrupt_ready <- disruption %>% tm( sampleId, gene, annotation = "disrupted", biallelic = NA, method = "DISRUPTION")
fusion_ready <- fusion %>% tm( sampleId, gene, annotation = "fusion", biallelic = NA, method = "FUSION")

- Only take top events on each chromosome for amplifications and deletions

In [ ]:
ct_threshold <- 10; 
rk_threshold <- 5

In [ ]:
amps_ready <- 
amp %>% 
 mu(arm = ifelse(grepl("q", chromosomeBand), "q", "p")) %>% 
 gb(gene) %>% mu(ct = n()) %>% ug() %>% 
 gb(chromosome, arm) %>% mu(rk = min_rank(desc(ct))) %>% ug() %>% 
 fi(ct > ct_threshold, rk <= rk_threshold) %>% 
 tm(sampleId, gene, annotation = "high_cn", biallelic = NA, method = "AMP")

In [ ]:
dels_ready <- 
del %>% 
 mu(arm = ifelse(grepl("q", chromosomeBand), "q", "p")) %>% 
 gb(gene) %>% mu(ct = n()) %>% ug() %>% 
 gb(chromosome, arm) %>% mu(rk = min_rank(desc(ct))) %>% ug() %>% 
 fi(ct > ct_threshold, rk <= rk_threshold) %>% 
 tm(sampleId, gene, annotation = "low_cn", biallelic = NA, method = "DEL")

- Together

In [ ]:
full_drivers <- 
rbind(somatic_ready, amps_ready, dels_ready, disrupt_ready, fusion_ready) %>% 
 mu(method = factor(method, levels = c("HOTSPOT", "BIALLELIC", "DNDS", "INFRAME", "DEL", "AMP", "DISRUPTION", "FUSION"))) %>% 
 gb(sampleId, gene) %>% su(method2 = paste0(unique(method), collapse = "_")) 

# 1 - Share full drivers

In [ ]:
fwrite(full_drivers, "/mnt/petasan_immunocomp/datasets/hartwig/biomarkers/database/drivers_full/full_drivers.txt")